In [16]:
import cv2
import numpy as np
import serial
import time

# 打开摄像机和端口，不轻易运行
# cap=cv2.VideoCapture(0)
# ser_stm32 = serial.Serial('/dev/ttyAMA0', 921600)
# ser_car2 = serial.Serial('/dev/ttyAMA3', 9600)

# 定义广义常亮
stop='V+000'
red_light_on='OKOK'
red_light_off='OFFF'
yellow_light_on='GOOD'
yellow_light_off='NONO'

# 在判断好数字后将决定前进的方向，输入为G±nn，L±nn，R±nn，到达终点后，发送OKOK
def send_order(order,ser):
    print(order)
    encoded_data = order.encode()
    ser.write(encoded_data)

# 发送命令给stm32
def send_stm32(order):
    send_order(order,ser_stm32)

# 发送命令给车2
def send_car2(order):
    send_order(order,ser_car2)

# 获得stm32的信号
def get_stm32():
    hex_data = ser_stm32.read(1)
    return hex_data

# 获取2车信号
def get_car2():
    hex_data = ser_car2.read(1)
    return hex_data

# 这一步用于识别黑色条纹的位置，计算误差后传给stm32
def get_road_error(road):
    d=np.argwhere(road[[450,350,250],:] > 250)
    dis=np.mean(d,axis=0)-320
    k=dis[1]
    k=k*20
    error='E+000'
    k=min(max(-10000,k),10000)
    x=0.8
    if k>=0:
        error='E'+'-'+str(int(int(k**(x))%1000/100))+str(int(int(k**(x))%100/10))+str(int(k**(x))%10)
    if k<0:
        error='E'+'+'+str(int(int((-k)**(x))%1000/100))+str(int(int((-k)**(x))%100/10))+str(int((-k)**(x))%10)
    # print(error)
    return error,k

# 对图片进行分割操作, 分别返回黑色与红色的图片
def photo_process(cap):
    _,image=cap.read()d=np.abs(image[:,:,2].astype(np.float32)-0.5*(image[:,:,0].astype(np.float32)+image[:,:,1].astype(np.float32)))
    black=np.abs((image[:,:,2].astype(np.float32)+image[:,:,0].astype(np.float32)+image[:,:,1].astype(np.float32))/3)
    # 对红色图片进行二值化，阈值为30
    _, red = cv2.threshold(red, 45, 255, cv2.THRESH_BINARY)
    _, black= cv2.threshold(black, 60, 255, cv2.THRESH_BINARY)
    return red, black

# 依据前方状态直行
def car_go_straight(road):
    error,_=get_road_error(road)
    #send_order(error,ser_stm32)

# 立即停止
def car_stop():
    time.sleep(0.1)
    send_order(stop,ser_stm32)
    time.sleep(0.1)

# 题目1速度0.3m/s
go1_low='V+28'
go1_high='V+32'
see_car2_last=0
# 依据前方状态直行，并调整速度
def car_go_straight_adjust(red, black, see_car2_last):
    error,_=get_road_error(black)
    send_order(error,ser_stm32)
    if (see_car2(red)==1)&(see_car2_last==0):
        send_car2(go1_low)
        see_car2_last=1
    if (see_car2(red)==0)&(see_car2_last==1):
        send_car2(go1_high)
        see_car2_last=0
    return see_car2_last

# 车往前冲
def car_rush():
    for i in range(10):
        send_stm32('E+000')

# 车转弯
def car_turn():
    for i in range(10):
        send_stm32('E+030')

In [2]:
# 1.测试路口弯道识别
# 在路口识别分支，有分支为0，正常为1

def get_cross(road):
    if np.sum(road[:,0:250] > 250)>300:
        return 0
    else:
        return 1

# 2.测试停车标志识别
# 监测停止标志, 有为0，没有为1
def get_stop(road):
    if (np.sum(road[:,0:250] > 250)>100)&(np.sum(road[:,389:639] > 250)>100):
        return 0
    else:
        return 1

# 3.测试车辆识别
# 看前面有没有2车，有为1，没有为0
def see_car2(red):
    if np.sum(red[0:240,200:439] > 250)>200:
        return 1
    else:
        return 0

In [3]:
cap1=cv2.VideoCapture(1)
# cap2=cv2.VideoCapture(2)

In [ ]:
while True:
    _, image1=cap1.read()
    # _, image2=cap2.read()
    cv2.imshow('image1',image1)
    # cv2.imshow('image2',image2)
    red1, black1 = photo_process(cap1)
    # red2, black2 = photo_process(cap2)
    cv2.imshow('black1',black1)
    cv2.imshow('red1',red1)
    cv2.waitKey(1)
    # print('1车是否看到弯道   ：'+str(np.sum(black1[:,0:250] < 250)))
    # print('2车是否看到弯道   ：'+str(get_cross(black2)))
    # print('1车是否看到停车   ：'+str(get_stop(black1)))
    # print('2车是否看到停车   ：'+str(get_stop(black2)))
    # print('1车是否看到车2    ：'+str(see_car2(red1)))
    # print('2车是否看到车1    ：'+str(see_car2(red2)))
    red, black=photo_process(cap1)
    car_go_straight(black)
    #time.sleep(0.1)